In [ ]:
import os
import numpy as np
import scipy.spatial.distance as dist
import matplotlib as mpl
import matplotlib.pyplot as plt
import gudhi

_tol = 1e-10

import IBloFunMatch_inter as ibfm
output_dir = "output" 

## Define a moving point cloud

In [ ]:
def take_sample(point_cloud, no_samples, seed=None):
    """ From PerMaViss.py
    """
    if seed != None:
        np.random.seed(seed)
    no_points = len(point_cloud)
    no_samples = min(no_points, no_samples)
    Dist = dist.squareform(dist.pdist(point_cloud))
    k = int(np.random.rand() * no_points) - 1
    selection = [k]
    point_selection = [point_cloud[k]]
    for i in range(no_samples-1):
        not_selected = np.setdiff1d(range(no_points), selection)
        rest_Dist = np.min(Dist[not_selected][:, selection], axis=1)
        k = not_selected[np.argmax(rest_Dist)]
        selection.append(k)
        point_selection.append(point_cloud[k])

    return np.array(point_selection)

In [ ]:
point_cloud = []
RandGen = np.random.default_rng(5)
C1 = ibfm.sampled_circle(4, 7, 40, RandGen)
num_samples = 15
C1 = take_sample(C1, num_samples, seed=4)
fig = plt.figure(figsize=(3,3))
ax = fig.add_axes((0,0,1,1))
# viridis_cm = mpl.colormaps["viridis"].resampled(num_samples)
viridis_cm = mpl.colormaps["viridis"]
colorlist = [viridis_cm(idx/num_samples) for idx in range(num_samples)]
ax.scatter(C1[:,0], C1[:,1], color=colorlist)
ax.set_frame_on(False)
ax.set_xticks([])
ax.set_yticks([])
plt.savefig("plots/fleet/points_fleet.png")

In [ ]:
Csmall = C1*0.7 + (RandGen.random(num_samples*2).reshape((num_samples, 2)) -[0.5, 0.5])*0.9 + [0.7,0]
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(6,3))
ax[0].scatter(C1[:,0], C1[:,1], color=colorlist, zorder=2)
ax[1].scatter(C1[:,0], C1[:,1], color=colorlist, alpha=0.05, zorder=1.5)
ax[1].scatter(Csmall[:,0], Csmall[:,1], color=colorlist, zorder=2)
# Draw trajectory lines
for ptA, ptB, color in zip(C1, Csmall, colorlist):
    new_line = mpl.lines.Line2D([ptA[0], ptB[0]], [ptA[1], ptB[1]], lw=2, color=color, zorder=1, alpha=0.2)
    ax[1].add_line(new_line)
# Set axis parameters
for ax_it in ax:
    ax_it.set_xticks([])
    ax_it.set_yticks([])
    ax_it.set_aspect("equal")
# end for 
ax[0].set_title("BEFORE")
ax[1].set_title("AFTER")
plt.savefig("plots/fleet/points_fleet.png")

In [ ]:
RandGen = np.random.default_rng(5)
C2 = C1*1.1 + (RandGen.random(num_samples*2).reshape((num_samples, 2)) -[0.5, 0.5])*3 + [0.7,0]
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(6,3))
ax[0].scatter(C1[:,0], C1[:,1], color=colorlist, zorder=2)
ax[1].scatter(C1[:,0], C1[:,1], color=colorlist, alpha=0.05, zorder=1.5)
ax[1].scatter(C2[:,0], C2[:,1], color=colorlist, zorder=2)
# Draw trajectory lines
for ptA, ptB, color in zip(C1, C2, colorlist):
    new_line = mpl.lines.Line2D([ptA[0], ptB[0]], [ptA[1], ptB[1]], lw=2, color=color, zorder=1, alpha=0.2)
    ax[1].add_line(new_line)
# Set axis parameters
for ax_it in ax:
    ax_it.set_xticks([])
    ax_it.set_yticks([])
    ax_it.set_aspect("equal")
# end for 
ax[0].set_title("BEFORE")
ax[1].set_title("AFTER")
plt.savefig("plots/fleet/points_fleet_2.png")

### Depict middle distance $d_M$ compared to $d_1$ and $d_2$

In [ ]:
import gudhi
import scipy.spatial.distance as dist

In [ ]:
def plot_complex(Dist_X, points, max_rad, ax):
    rips_complex = gudhi.RipsComplex(distance_matrix=Dist_X, max_edge_length=max_rad)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
    for filtered_value in simplex_tree.get_filtration():
        spx = filtered_value[0]
        if len(spx)==2:
            edge = points[spx]
            ax.add_line(mpl.lines.Line2D(edge[:,0], edge[:,1], lw=3, color="black", zorder=1))
        elif len(spx)==3:
            triangle = points[spx] 
            ax.add_patch(mpl.patches.Polygon(triangle, facecolor="black", alpha=0.3, zorder=0.5))
        # end if else 
    # end for 
# def plot_complex

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(12,4))
max_rad =5.5
point_list = [C1, (C1+C2)/2, C2]
# Compute Distance Matrices
Dist_C1 = dist.squareform(dist.pdist(C1))
Dist_C2 = dist.squareform(dist.pdist(C2))
Dist_M = np.maximum(Dist_C1, Dist_C2)
Dist_list = [Dist_C1, Dist_M, Dist_C2]
title_list = [f"$d_1 \leq {max_rad:.1f}$", f"$d_M \leq {max_rad:.1f}$", f"$d_2 \leq {max_rad:.1f}$"]
for idx, (X, Dist_X) in enumerate(zip(point_list, Dist_list)):
    ax[idx].scatter(X[:,0], X[:,1], zorder=2, color=colorlist, s=200)
    plot_complex(Dist_X, X, max_rad, ax[idx])
    ax[idx].set_aspect("equal")
    ax[idx].set_frame_on(False)
    ax[idx].set_xticks([])
    ax[idx].set_yticks([])
    ax[idx].set_title(title_list[idx], fontsize=30)
# end for 
fig.tight_layout()
plt.savefig("plots/fleet/VR_cpx_Middle.png")

## Example 1:

In [ ]:
%%capture
# Recompute point cloud
RandGen = np.random.default_rng(6)
num_samples, low_rad, big_rad = 40, 4, 7
radii = RandGen.uniform(low_rad, big_rad, num_samples)
angles = RandGen.uniform(0,2*np.pi, num_samples)
C1 = np.array([np.cos(angles)*radii, np.sin(angles)*radii]).T
num_samples = 15
C1 = take_sample(C1, num_samples, seed=4)
C2 = C1*0.6 + (RandGen.random(num_samples*2).reshape((num_samples, 2)) -[0.5, 0.5])*1.5 + [1,0]
C1 += (RandGen.random(num_samples*2).reshape((num_samples, 2)) -[0.5, 0.5])*1.5
# Compute Distance Matrices
Dist_C1 = dist.squareform(dist.pdist(C1))
Dist_C2 = dist.squareform(dist.pdist(C2))
Dist_M = np.maximum(Dist_C1, Dist_C2)
assert(np.all(Dist_C1 <= Dist_M))
assert(np.all(Dist_C2 <= Dist_M))
idS = [i for i in range(Dist_M.shape[0])]
# Initialize variables to range over 
output_match_C1 = ibfm.get_IBloFunMatch_subset(Dist_M, Dist_C1, idS, output_dir)
output_match_C2 = ibfm.get_IBloFunMatch_subset(Dist_M, Dist_C2, idS, output_dir)
# Prepare intervals for plotting
persistence = [(0, list(bar)) for bar in output_match_C1["X_barcode"]]
persistence += [(1, list(bar)) for bar in output_match_C2["X_barcode"]]
persistence += [(2, list(bar)) for bar in output_match_C1["S_barcode"]]
max_rad = max(np.max(output_match_C1["S_barcode"]), np.max(output_match_C1["X_barcode"]), np.max(output_match_C2["X_barcode"]))
persistence += [(3, [0, max_rad])]

In [ ]:
# Plot
colorlist = ["red", "green", "blue", "white"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(3,3))
gudhi.plot_persistence_diagram(persistence, axes=ax, colormap=colorlist)
# Plot matchings 
for idx_output, output_match in enumerate([output_match_C1, output_match_C2]):
    X_barcode = output_match["X_barcode"]
    S_barcode = output_match["S_barcode"]
    for idx, idx_match in enumerate(output_match["induced_matching"]):
        if idx_match<0:
            continue
        edge = np.array([S_barcode[idx],X_barcode[idx_match]])
        ax.plot(edge[:,0], edge[:,1], color=colorlist[idx_output], linewidth=2, zorder=0.5)
ax.get_legend().set_visible(False)
fig.tight_layout()
plt.savefig("plots/fleet/Matchings_fleet.png")

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(12,4))
max_rad =5.5
point_list = [C1, (C1+C2)/2, C2]
# Compute Distance Matrices
Dist_C1 = dist.squareform(dist.pdist(C1))
Dist_C2 = dist.squareform(dist.pdist(C2))
Dist_M = np.maximum(Dist_C1, Dist_C2)
Dist_list = [Dist_C1, Dist_M, Dist_C2]
colorlist = [viridis_cm(idx/Dist_M.shape[0]) for idx in range(Dist_M.shape[0])]
title_list = [f"$d_1 \leq {max_rad:.1f}$", f"$d_M \leq {max_rad:.1f}$", f"$d_2 \leq {max_rad:.1f}$"]
for idx, (X, Dist_X) in enumerate(zip(point_list, Dist_list)):
    ax[idx].scatter(X[:,0], X[:,1], zorder=2, color=colorlist, s=200)
    plot_complex(Dist_X, X, max_rad, ax[idx])
    ax[idx].set_aspect("equal")
    ax[idx].set_frame_on(False)
    ax[idx].set_xticks([])
    ax[idx].set_yticks([])
    ax[idx].set_title(title_list[idx], fontsize=30)
# end for 
ax[2].scatter(C1[:,0], C1[:,1], color=colorlist, s=200, alpha=0.1)
fig.tight_layout()
plt.savefig("plots/fleet/VR_cpx_match_first.png")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3,3))
ax.scatter(C1[:,0], C1[:,1], color=colorlist, alpha=0.05, zorder=1.5)
ax.scatter(C2[:,0], C2[:,1], color=colorlist, zorder=2)
# Draw trajectory lines
for ptA, ptB, color in zip(C1, C2, colorlist):
    new_line = mpl.lines.Line2D([ptA[0], ptB[0]], [ptA[1], ptB[1]], lw=2, color=color, zorder=1, alpha=0.2)
    ax.add_line(new_line)
# Set axis parameters
ax.set_xticks([])
ax.set_yticks([])
ax.set_aspect("equal")
# end for 
plt.savefig("plots/fleet/points_move_first.png")

## Example 2

In [ ]:
%%capture
# Recompute point cloud
RandGen = np.random.default_rng(5)
num_samples, low_rad, big_rad = 40, 4, 7
radii = RandGen.uniform(low_rad, big_rad, num_samples)
angles = RandGen.uniform(0,2*np.pi, num_samples)
C1 = np.array([np.cos(angles)*radii, np.sin(angles)*radii]).T
num_samples = 15
C1 = take_sample(C1, num_samples, seed=4)
C2 = C1 + (RandGen.random(num_samples*2).reshape((num_samples, 2)) -[0.5, 0.5])*3 + [1,0]
C1 += (RandGen.random(num_samples*2).reshape((num_samples, 2)) -[0.5, 0.5])*3
# Compute Distance Matrices
Dist_C1 = dist.squareform(dist.pdist(C1))
Dist_C2 = dist.squareform(dist.pdist(C2))
Dist_M = np.maximum(Dist_C1, Dist_C2)
assert(np.all(Dist_C1 <= Dist_M))
assert(np.all(Dist_C2 <= Dist_M))
idS = [i for i in range(Dist_M.shape[0])]
output_match_C1 = ibfm.get_IBloFunMatch_subset(Dist_M, Dist_C1, idS, output_dir)
output_match_C2 = ibfm.get_IBloFunMatch_subset(Dist_M, Dist_C2, idS, output_dir)
# Prepare intervals for plotting
persistence = [(0, list(bar)) for bar in output_match_C1["X_barcode"]]
persistence += [(1, list(bar)) for bar in output_match_C2["X_barcode"]]
persistence += [(2, list(bar)) for bar in output_match_C1["S_barcode"]]
max_rad = max(np.max(output_match_C1["S_barcode"]), np.max(output_match_C1["X_barcode"]), np.max(output_match_C2["X_barcode"]))
persistence += [(3, [0, max_rad])]

In [ ]:
# Plot
colorlist = ["red", "green", "blue", "white"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(3,3))
gudhi.plot_persistence_diagram(persistence, axes=ax, colormap=colorlist)
# Plot matchings 
for idx_output, output_match in enumerate([output_match_C1, output_match_C2]):
    X_barcode = output_match["X_barcode"]
    S_barcode = output_match["S_barcode"]
    for idx, idx_match in enumerate(output_match["induced_matching"]):
        if idx_match<0:
            continue
        edge = np.array([S_barcode[idx],X_barcode[idx_match]])
        ax.plot(edge[:,0], edge[:,1], color=colorlist[idx_output], linewidth=2, zorder=0.5)
ax.get_legend().set_visible(False)
fig.tight_layout()
plt.savefig("plots/fleet/Matchings_fleet_second.png")

In [ ]:
persistence

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(12,4))
max_rad =5.5
point_list = [C1, (C1+C2)/2, C2]
# Compute Distance Matrices
Dist_C1 = dist.squareform(dist.pdist(C1))
Dist_C2 = dist.squareform(dist.pdist(C2))
Dist_M = np.maximum(Dist_C1, Dist_C2)
Dist_list = [Dist_C1, Dist_M, Dist_C2]
colorlist = [viridis_cm(idx/Dist_M.shape[0]) for idx in range(Dist_M.shape[0])]
title_list = [f"$d_1 \leq {max_rad:.1f}$", f"$d_M \leq {max_rad:.1f}$", f"$d_2 \leq {max_rad:.1f}$"]
for idx, (X, Dist_X) in enumerate(zip(point_list, Dist_list)):
    ax[idx].scatter(X[:,0], X[:,1], zorder=2, color=colorlist, s=200)
    plot_complex(Dist_X, X, max_rad, ax[idx])
    ax[idx].set_aspect("equal")
    ax[idx].set_frame_on(False)
    ax[idx].set_xticks([])
    ax[idx].set_yticks([])
    ax[idx].set_title(title_list[idx], fontsize=30)
# end for 

ax[2].scatter(C1[:,0], C1[:,1], color=colorlist, s=200, alpha=0.1)
fig.tight_layout()
plt.savefig("plots/fleet/VR_cpx_match_second.png")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3,3))
ax.scatter(C1[:,0], C1[:,1], color=colorlist, alpha=0.05, zorder=1.5)
ax.scatter(C2[:,0], C2[:,1], color=colorlist, zorder=2)
# Draw trajectory lines
for ptA, ptB, color in zip(C1, C2, colorlist):
    new_line = mpl.lines.Line2D([ptA[0], ptB[0]], [ptA[1], ptB[1]], lw=2, color=color, zorder=1, alpha=0.2)
    ax.add_line(new_line)
# Set axis parameters
ax.set_xticks([])
ax.set_yticks([])
ax.set_aspect("equal")
# end for 
plt.savefig("plots/fleet/points_move_second.png")